In [80]:
import _init_paths
from datasets.factory import get_imdb
import cv2
import visdom
import numpy as np
from PIL import Image
import roi_data_layer.roidb as rdl_roidb
# vis = visdom.Visdom(server='http://localhost',port='8097')
imdb = get_imdb('voc_2007_test')
roidb = imdb.roidb
print len(roidb)


voc_2007_test ss roidb loaded from /home/studio/Documents/aman/crystal_test/hw2-release/code/data/cache/voc_2007_test_selective_search_roidb.pkl
4952


In [9]:
import torch.nn as nn
conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3)
nn.init.xavier_uniform(conv1.weight, gain = 0.01)

Parameter containing:
(0 ,0 ,.,.) = 
 -2.1049e-04  6.6547e-05 -3.1677e-04  ...  -3.3228e-04  7.3268e-05  2.3617e-04
 -7.7707e-05  3.1020e-04  3.5674e-04  ...   6.5325e-05  3.2835e-04 -2.9695e-04
  1.9465e-04 -5.9384e-05  2.4342e-04  ...   3.6542e-04  4.1211e-04 -3.7777e-04
                 ...                   ⋱                   ...                
  3.7603e-04 -1.8262e-04  3.8598e-04  ...  -1.9118e-05 -2.3636e-04  1.8963e-04
 -2.1106e-04  2.6598e-04  2.9887e-04  ...  -3.0545e-04 -2.9045e-04 -5.4433e-05
  3.1996e-04  4.2728e-04  1.2698e-04  ...   3.1989e-04  4.2035e-04 -4.8809e-05

(0 ,1 ,.,.) = 
  2.0352e-04 -2.9750e-04 -4.1229e-04  ...   2.6148e-04  1.0711e-04 -8.6606e-05
  9.3315e-05  1.6197e-05 -2.5884e-05  ...  -3.4580e-04  4.2347e-05  5.7261e-06
  1.2610e-04 -2.5333e-04 -1.7480e-04  ...   2.6166e-04 -3.3674e-04 -2.3033e-04
                 ...                   ⋱                   ...                
 -2.3042e-04 -1.1089e-04  3.5847e-04  ...   1.7666e-04 -1.4401e-04 -1.3505e-04

In [65]:
import torch.nn as nn
import torch
from torch.autograd import Variable
m = nn.Softmax(dim = 1)
n = nn.Softmax(dim = 0)
input = Variable(torch.randn(2, 3))
output = m(input)
output2 = n(input)
print output2
print output
tmp = torch.mul(output, output2).sum(0)
assert tmp.max().data.cpu().numpy() < 1
print tmp

Variable containing:
 0.6848  0.6947  0.5949
 0.3152  0.3053  0.4051
[torch.FloatTensor of size 2x3]

Variable containing:
 0.5711  0.2202  0.2088
 0.5239  0.1929  0.2833
[torch.FloatTensor of size 2x3]

Variable containing:
 0.5562
 0.2118
 0.2390
[torch.FloatTensor of size 3]



In [33]:
tmp[0]*tmp[1]

Variable containing:
1.00000e-02 *
  4.4717
[torch.FloatTensor of size 1]

In [53]:
def multilabel_loss(fx,y):
    loss = 0.0
    batch = fx.size(0)
    cls = fx.size(1)
#     print "{0}_batchs_{1}_cls".format(batch, cls)
    for b in range(batch):
        for k in range(cls):
            loss += torch.log(1+ torch.exp(-y[b,k]*fx[b,k]))
    loss = loss/batch
    return loss
def multilabel_loss2(fx,y):
    batch = fx.size(0)
    loss = torch.log(1+ torch.exp(-torch.mul(y,fx))).sum(0).sum(0)/batch
    return loss
input = Variable(torch.randn(10, 20), requires_grad=True)
input2 = Variable(input.data.clone(), requires_grad = True)
target = Variable(torch.randn(10, 20))
loss1 = multilabel_loss(input, target)
loss2 = multilabel_loss2(input2, target)
loss1.backward()
print input2.grad
loss2.backward()
print loss1
print loss2
print input2.grad
print input.grad



None
Variable containing:
 15.7933
[torch.FloatTensor of size 1]

Variable containing:
 15.7933
[torch.FloatTensor of size 1]

Variable containing:

Columns 0 to 9 
 0.0028 -0.0238 -0.0164  0.0085  0.0095  0.0126 -0.0503 -0.0236 -0.0306 -0.0887
-0.0603  0.1898  0.0402  0.0019 -0.0267  0.0382  0.0495 -0.0236  0.0035  0.0132
-0.0155  0.0310  0.0354  0.0227  0.0020  0.0329 -0.0275 -0.0050  0.0887  0.0227
 0.0045 -0.0221 -0.0054 -0.0546 -0.0377 -0.1233 -0.1099  0.0118  0.0235  0.0039
-0.0216 -0.0336 -0.0375  0.0958  0.0023  0.0495 -0.0117 -0.0180 -0.0093 -0.0240
-0.0021 -0.0210 -0.0520 -0.0143  0.0289 -0.0077 -0.0277  0.0453  0.0330 -0.0460
 0.0160  0.0072  0.0436 -0.0298  0.0665  0.0637  0.0259 -0.0244  0.0285  0.0373
-0.0212  0.0278 -0.0108  0.0524 -0.0270  0.0331 -0.0222  0.0143 -0.0054  0.0072
 0.0624 -0.0371  0.0092  0.0177 -0.0076 -0.0559 -0.0289 -0.0030  0.0203  0.0700
 0.0306  0.0598 -0.0639  0.0274 -0.0348  0.0012 -0.0792  0.0235 -0.0228 -0.0333

Columns 10 to 19 
 0.0254  0.0906 

In [60]:
num_classes = 20
labels_blob = np.zeros((0,num_classes), dtype=np.float32)
print labels_blob
labels = np.zeros((1, num_classes))-1
print labels.shape
labels_blob = np.vstack((labels, labels))
print labels_blob.shape

[]
(1, 20)
(2, 20)


In [73]:
def build_loss(cls_prob, label_vec):
    """Computes the loss

    :cls_prob: N_roix20 output scores
    :label_vec: 1x20 one hot label vector 
    :returns: loss

    """
    #TODO: Compute the appropriate loss using the cls_prob that is the
    #output of forward()
    #Checkout forward() to see how it is called
    ## alert, the image number should be 1 per batch
    loss = 0.0
    print "why the loss is negative?!"
    print label_vec
    n_classes = 10
    print cls_prob
    for cls_idx in range(n_classes):
        loss += torch.log(label_vec[cls_idx]*(cls_prob[cls_idx]-0.5)+0.5)

    return loss
input = Variable(torch.FloatTensor(10).uniform_(0, 1), requires_grad=True)
target = Variable(torch.FloatTensor([1,-1,1,1,1,-1,1,1,1,1]))
loss = build_loss(input, target)
print loss

why the loss is negative?!
Variable containing:
 1
-1
 1
 1
 1
-1
 1
 1
 1
 1
[torch.FloatTensor of size 10]

Variable containing:
 0.6682
 0.9996
 0.2944
 0.9321
 0.0584
 0.7651
 0.8253
 0.0476
 0.6402
 0.2346
[torch.FloatTensor of size 10]

Variable containing:
-18.8526
[torch.FloatTensor of size 1]



In [76]:
from datasets.factory import get_imdb
imdb = get_imdb('voc_2007_trainval')
print imdb
roidb = imdb.gt_roidb()
print roidb
print roidb[0]

voc_2007_trainval gt roidb loaded from /home/studio/Documents/aman/crystal_test/hw2-release/code/data/cache/voc_2007_trainval_gt_roidb.pkl
[{'boxes': array([[262, 210, 323, 338],
       [164, 263, 252, 371],
       [240, 193, 294, 298]], dtype=uint16), 'gt_classes': array([9, 9, 9], dtype=int32), 'gt_overlaps': <3x21 sparse matrix of type '<type 'numpy.float32'>'
	with 3 stored elements in Compressed Sparse Row format>, 'seg_areas': array([7998., 9701., 5830.], dtype=float32), 'flipped': False}, {'boxes': array([[140,  49, 499, 329]], dtype=uint16), 'gt_classes': array([7], dtype=int32), 'gt_overlaps': <1x21 sparse matrix of type '<type 'numpy.float32'>'
	with 1 stored elements in Compressed Sparse Row format>, 'seg_areas': array([101160.], dtype=float32), 'flipped': False}, {'boxes': array([[ 68, 171, 269, 329],
       [149, 140, 228, 283],
       [284, 200, 326, 330],
       [257, 197, 296, 328]], dtype=uint16), 'gt_classes': array([13, 15, 15, 15], dtype=int32), 'gt_overlaps': <4x21